### Imports

In [1]:
import spotpy
import pandas as pd

from mogestpy.quantity.Hydrological import SMAP
from spotpy.objectivefunctions import mae, rmse, nashsutcliffe, kge, pbias

Pandas Plotting Backend

In [2]:
pd.options.plotting.backend = "plotly"

Loading data into a Pandas DataFrame

In [3]:
df = pd.read_excel(r'D:\Dados\smap_jundiai.xlsx')
display(df)

,ETP,P,Qobs,Qplan
0,5.266667,0.000000,1.69,-0.323000
1,5.266667,0.000000,0.06,-0.319765
2,5.266667,1.100000,0.41,-0.316562
3,5.266667,0.000000,0.06,-0.313391
4,5.266667,0.000000,1.94,-0.310251
...,...,...,...,...
2118,3.233333,0.000000,0.11,0.028231
2119,3.233333,0.283333,0.09,0.025673
2120,3.233333,0.003333,0.02,0.024410
2121,3.233333,0.496667,0.01,0.023720


In [4]:
bacia = SMAP.SMAP.Basin(AD=109.08)
ponto = SMAP.SMAP.Point(df['P'], df['ETP'])
modelo = SMAP.SMAP(ponto, bacia)

In [5]:
result = modelo.Calibrate(df.Qobs)

In [6]:
print(bacia)

SMAP Basin Object:
  Str = 1049.9999988026211,
  Crec = 9.999999987396013,
  Capc = 39.999999987396016,
  kkt = 104.99999990547009,
  k2t = 0.2,
  Ai = 3.500000008109402,
  TUin = 0.49999999936980066,
  EBin = 0.0,
  AD = 109.08


In [7]:
df['Qcodigo'] = modelo.Q

In [ ]:
def teste_fo(eval, Q):
    v_nse = - nashsutcliffe(eval, Q)
    v_kge = - kge(eval, Q)
    v_pbias = pbias(eval, Q)
    
    return v_nse + v_kge + v_pbias

In [ ]:
# result2 = modelo.Calibrate(df['Qobs'], objective_function=teste_fo, optimization_engine='de')

In [ ]:
df['Qfo-teste'] = modelo.Q

In [ ]:
def funcao_objetivo(eval, sim):
    somatorio = 0
    for i, j in zip(eval, sim):
        somatorio += (i - j) ** 2
    return somatorio

In [ ]:
result_test = modelo.Calibrate(df.Qobs, objective_function=funcao_objetivo, optimization_engine='differential_evolution')

In [ ]:
result_test

In [ ]:
print(bacia)

In [ ]:
df['Qcustom_fo'] = modelo.Q

In [ ]:
bacia2=SMAP.SMAP.Basin(
    Str = 235.00569097847588,
    Crec = 11.489898491508313,
    TUin = 0.8544213137461782,
    EBin = 10.965839303606261,
    Capc = 40.02608612021428,
    kkt = 79.03366335608037,
    k2t = 0.9532324848266956,
    Ai = 2.0253907074478206,
    AD=109.08
)

In [ ]:
modelo2 = SMAP.SMAP(ponto, bacia2)

In [ ]:
modelo2.Calibrate(df.Qobs, objective_function='kge')

In [ ]:
df['Calib_DE'] = modelo2.Q

In [ ]:
df[['Qobs', 'Qplan', 'Qcodigo', 'Qcustom_fo']].plot()

Calibrador

In [ ]:
bacia = SMAP.SMAP.Basin(
    Str=100,
    k2t=0.848790939326689,
    Crec=1.00998050978914,
    Ai=2.44891091946412,
    Capc=50,
    kkt=68.85081378162,
    TUin=5/100,
    EBin=0,
    AD=109.08
)
ponto = SMAP.SMAP.Point(df['P'], df['ETP'])
modelo = SMAP.SMAP(ponto, bacia)
modelo.RunModel()

kge = spotpy.objectivefunctions.kge(df['Qobs'], modelo.Q)

print(kge)

In [ ]:
df

In [ ]:
df['Qcod'] = modelo.Q

df.filter(items=['Qcod', 'Qobs']).plot()


In [ ]:
# import spotpy
# import scipy.optimize
# from scipy.optimize import differential_evolution
# scipy.optimize.options={'disp': True}

# precipitation = df.P.to_list()
# evapotranspiration = df.ETP.to_list()
# observed_discharge = df.Qobs

# bounds = [[100.0, 2000.0],  # Str              
#               [0.0, 20],    # Crec
#               [0.0, 1.0],   # TUin
#               [0.0, 20],    # EBin
#               [30, 50],     # Capc
#               [30, 180],    # kkt
#               [0.2, 10],    # k2t
#               [2, 5]]       # Ai

# def objective(p):
#     Str, Crec, TUin, EBin, Capc, kkt, k2t, Ai = p

#     bacia = SMAP.SMAP.Basin(
#         Str=Str,
#         k2t=k2t,
#         Crec=Crec,
#         Ai=Ai,
#         Capc=Capc,
#         kkt=kkt,
#         TUin=TUin,
#         EBin=EBin,
#         AD=109.08
#         )
    
#     ponto = SMAP.SMAP.Point(precipitation, evapotranspiration)
#     modelo = SMAP.SMAP(ponto, bacia)
#     modelo.RunModel()
    
#     return -spotpy.objectivefunctions.kge(observed_discharge, modelo.Q)


# result = differential_evolution(objective, bounds, maxiter=1000)

# for i in result['x']:
#     print(i)

In [ ]:
# import spotpy
# from scipy.optimize import minimize

# precipitation = df.P.to_list()
# evapotranspiration = df.ETP.to_list()
# observed_discharge = df.Qobs

# bounds = [[100.0, 2000.0],  # Str              
#               [0.0, 20],    # Crec
#               [0.0, 1.0],   # TUin
#               [0.0, 20],    # EBin
#               [30, 50],     # Capc
#               [30, 180],    # kkt
#               [0.2, 10],    # k2t
#               [2, 5]]       # Ai

# def objective(p):
#     Str, Crec, TUin, EBin, Capc, kkt, k2t, Ai = p

#     bacia = SMAP.SMAP.Basin(
#         Str=Str,
#         k2t=k2t,
#         Crec=Crec,
#         Ai=Ai,
#         Capc=Capc,
#         kkt=kkt,
#         TUin=TUin,
#         EBin=EBin,
#         AD=109.08
#         )
    
#     ponto = SMAP.SMAP.Point(precipitation, evapotranspiration)
#     modelo = SMAP.SMAP(ponto, bacia)
#     modelo.RunModel()

#     return -spotpy.objectivefunctions.kge(observed_discharge, modelo.Q)


# result = minimize(objective, x0=[100, 0, 0, 0, 39, 39, .2, 2], bounds=bounds)

# for i, j in zip(result['x'], ['Str', 'Crec', 'TUin', 'EBin', 'Capc', 'kkt', 'k2t', 'Ai']):
#     print(f'{j} = {i},')

In [ ]:
result = modelo.Calibrate(df.Qobs)

In [ ]:
print(bacia)

In [ ]:
for i, j in zip(result['x'], ['Str', 'Crec', 'TUin', 'EBin', 'Capc', 'kkt', 'k2t', 'Ai']):
    print(f'{j} = {i},')

In [ ]:
modelo.RunModel()

In [ ]:
df['Teste'] = modelo.Q

In [ ]:
pd.options.plotting.backend = "plotly"

df[['Qobs','Teste', 'Qplan']].plot()

In [ ]:
result = modelo.Calibrate(df.Qobs, objective_function='kge')
print(bacia)

In [ ]:
df['Teste2'] = modelo.Q

In [ ]:
df[['Qobs','Teste', 'Teste2', 'Qplan']].plot()

In [ ]:
print(bacia)

In [ ]:
modelo.Calibrate(df.Qobs, objective_function='pbias')['x']

In [ ]:
print(bacia)

In [ ]:
df['teste12'] = modelo.Q
df[['Qobs','Teste', 'Teste2', 'Qplan', 'teste12']].plot()